In [1]:
import torch

from sentence_transformers import SentenceTransformer, CrossEncoder, util
from transformers import (AutoModelForCausalLM, AutoTokenizer, pipeline,
                          BitsAndBytesConfig, TrainingArguments, Trainer,
                          DataCollatorForLanguageModeling)
from datasets import load_dataset
import datasets
from pathlib import Path
datasets.config.DOWNLOADED_DATASETS_PATH = Path('/home/hpc/b110dc/b110dc16/datasets')

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from huggingface_hub import login
import wandb

import hnswlib

import os
import matplotlib.pyplot as plt
import re
import numpy as np
from tqdm import tqdm

from gensim.models.word2vec import Word2Vec

from IPython.display import HTML, Audio
# from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import scipy
import io
import ffmpeg

import whisper

In [2]:
ds = load_dataset("medalpaca/medical_meadow_medical_flashcards")
arr_len = np.array([ len(arr) for arr in ds['train']['input'] ], dtype=np.int16)
filter_arr = np.argwhere(arr_len > 2).reshape(-1,)

ds['train'] = ds['train'].select(filter_arr)
print(ds)

KeyboardInterrupt: 

In [ ]:
index_path = './hnswlib_filter.index'
corpus = np.array(ds['train']['output'])
print("Creating a HNSWLIB index")
semb_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
corpus_embeddings = semb_model.encode(corpus, convert_to_tensor=True,
                                    show_progress_bar=True)
index = hnswlib.Index(space='cosine', dim=corpus_embeddings.size(1))
index.init_index(max_elements=corpus_embeddings.size(0),
               ef_construction=256, M=64)
index.add_items(corpus_embeddings.cpu(), list(range(len(corpus_embeddings))))
print("Saving index to:", index_path)
index.save_index(index_path)